In [ ]:
!pip install cfgrib xarray matplotlib pandas
!pip install dask

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import cfgrib

### ERA5_Monthly_averaged_reanalysis_by_hour_of-day_data_on_single_levels_from_2013-2023.grib

#### Combine Datasets

In [173]:
file_path = r"C:\Users\giann\OneDrive\Desktop\Thesis\Copernicus_Data\ERA5_Monthly_averaged_reanalysis_by_hour_of-day_data_on_single_levels_from_2013-2023.grib"

# Replace backslashes with forward slashes
updated_path = file_path.replace('\\', '/')

# Print the updated path
print("Original Path:", file_path)
print("Updated Path:", updated_path)

# Use open_datasets and NO open_dataset(from Error)
ds = cfgrib.open_datasets(updated_path)
print(ds)


Original Path: C:\Users\giann\OneDrive\Desktop\Thesis\Copernicus_Data\ERA5_Monthly_averaged_reanalysis_by_hour_of-day_data_on_single_levels_from_2013-2023.grib
Updated Path: C:/Users/giann/OneDrive/Desktop/Thesis/Copernicus_Data/ERA5_Monthly_averaged_reanalysis_by_hour_of-day_data_on_single_levels_from_2013-2023.grib
[<xarray.Dataset> Size: 220MB
Dimensions:              (time: 528, latitude: 153, longitude: 341)
Coordinates:
    number               int32 4B 0
  * time                 (time) datetime64[ns] 4kB 2013-01-01 ... 2023-12-01T...
    step                 timedelta64[ns] 8B 00:00:00
    depthBelowLandLayer  float64 8B 0.0
  * latitude             (latitude) float64 1kB 72.0 71.75 71.5 ... 34.25 34.0
  * longitude            (longitude) float64 3kB -25.0 -24.75 ... 59.75 60.0
    valid_time           (time) datetime64[ns] 4kB 2013-01-01 ... 2023-12-01T...
Data variables:
    swvl1                (time, latitude, longitude) float32 110MB ...
    stl1                 (time, lati

In [179]:
print(type(ds),len(ds))
ds[0]

<class 'list'> 4


<xarray.Dataset> Size: 220MB
Dimensions:              (time: 528, latitude: 153, longitude: 341)
Coordinates:
    number               int32 4B 0
  * time                 (time) datetime64[ns] 4kB 2013-01-01 ... 2023-12-01T...
    step                 timedelta64[ns] 8B 00:00:00
    depthBelowLandLayer  float64 8B 0.0
  * latitude             (latitude) float64 1kB 72.0 71.75 71.5 ... 34.25 34.0
  * longitude            (longitude) float64 3kB -25.0 -24.75 ... 59.75 60.0
    valid_time           (time) datetime64[ns] 4kB 2013-01-01 ... 2023-12-01T...
Data variables:
    swvl1                (time, latitude, longitude) float32 110MB 0.3908 ......
    stl1                 (time, latitude, longitude) float32 110MB 250.6 ... ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

#### Data Cleaning

In [189]:
# Loop through each dataset in the list 'ds'
for idx, dataset in enumerate(ds):
    # Check for missing or NaN values in the current dataset
    missing_values = dataset.isnull().sum()
    print(f"Missing values in dataset {idx + 1}:")
    print(missing_values)


Missing values in dataset 1:
<xarray.Dataset> Size: 28B
Dimensions:              ()
Coordinates:
    number               int32 4B 0
    step                 timedelta64[ns] 8B 00:00:00
    depthBelowLandLayer  float64 8B 0.0
Data variables:
    swvl1                int32 4B 0
    stl1                 int32 4B 0
Missing values in dataset 2:
<xarray.Dataset> Size: 24B
Dimensions:              ()
Coordinates:
    number               int32 4B 0
    step                 timedelta64[ns] 8B 00:00:00
    depthBelowLandLayer  float64 8B 7.0
Data variables:
    swvl2                int32 4B 0
Missing values in dataset 3:
<xarray.Dataset> Size: 44B
Dimensions:  ()
Coordinates:
    number   int32 4B 0
    step     timedelta64[ns] 8B 00:00:00
    surface  float64 8B 0.0
Data variables:
    slt      int32 4B 0
    sp       int32 4B 0
    u10      int32 4B 0
    v10      int32 4B 0
    t2m      int32 4B 0
    d2m      int32 4B 0
Missing values in dataset 4:
<xarray.Dataset> Size: 32B
Dimensions:  (

### Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2013-2020

#### Combine Datasets

In [190]:

folder_path = r"C:\Users\giann\OneDrive\Desktop\Thesis\Copernicus_Data\Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2013-2020"

# Find all .nc files in the folder
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")]

# Fix the 'step' dimension and ensure it's unique
def preprocess(ds):
    if "step" in ds.coords and "valid_time" in ds.coords:
        # Replace 'step' with 'valid_time'
        ds = ds.assign_coords(step=("step", ds["valid_time"].values))
    
    # Deduplicate or aggregate 'step'
    if "step" in ds.coords:
        # Remove duplicates, keeping the first occurrence
        _, index = np.unique(ds["step"], return_index=True)
        # ds = ds.isel(step=index) 
        ds = ds.groupby("step").mean()
    return ds

# Categorize files by member size
files_by_member_size = {25: [], 51: []}

for file in file_list:
    ds = xr.open_dataset(file)
    member_size = ds.sizes.get('member', None)
    if member_size == 25:
        files_by_member_size[25].append(file)
    elif member_size == 51:
        files_by_member_size[51].append(file)

# Combine datasets
combined_datasets = {}

for member_size, files in files_by_member_size.items():
    if files:
        # Use open_mfdataset to process files and reduce memory usage
        
        # Chunking divides the dataset into manageable parts based on one or more dimensions
        # Each "chunk" is a smaller portion of the dataset, and operations are performed on these
        # chunks sequentially or in parallel, which helps reduce memory usage and allows processing 
        # of datasets that are too large to fit into RAM.
        ds = xr.open_mfdataset(
            files,
            preprocess=preprocess,
            combine="nested", #assumes that the files are ordered logically in the same way as the desired concatenation order
            concat_dim="member",
            chunks={"step": 10}  
        )
        
        # Sort the dataset by 'step' in ascending order
        ds = ds.sortby("step", ascending=True)
        
        combined_datasets[member_size] = ds
        print(f"Combined dataset with {member_size} members:")
        print(ds)
    else:
        print(f"No datasets found with {member_size} members.")


Combined dataset with 25 members:
<xarray.Dataset> Size: 1TB
Dimensions:                       (step: 54, y: 950, x: 1000, member: 1200)
Coordinates:
  * y                             (y) float64 8kB 5.498e+06 ... 7.525e+05
  * x                             (x) float64 8kB 2.502e+06 ... 7.498e+06
  * step                          (step) datetime64[ns] 432B 2013-01-31 ... 2...
    number                        (member) int64 10kB dask.array<chunksize=(25,), meta=np.ndarray>
Dimensions without coordinates: member
Data variables:
    rdis                          (step, y, x, member) float32 246GB dask.array<chunksize=(1, 950, 1000, 25), meta=np.ndarray>
    latitude                      (member, step, y, x) float32 246GB dask.array<chunksize=(25, 1, 950, 1000), meta=np.ndarray>
    longitude                     (member, step, y, x) float32 246GB dask.array<chunksize=(25, 1, 950, 1000), meta=np.ndarray>
    lambert_azimuthal_equal_area  (member, step) float64 518kB -2.147e+09 ......
    l

### Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2021-2023

#### See Dimension of Dataset

In [167]:

folder_path = r"C:\Users\giann\OneDrive\Desktop\Thesis\Copernicus_Data\Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2021-2023"

# Get the list of all .nc files in the folder
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".nc")]

# Go through each file and show its dimensions
for idx, file_path in enumerate(file_list):
    try:
        # Open the dataset
        ds = xr.open_dataset(file_path)
        
        # Print the file name and its dimensions
        print(f"Dataset {idx + 1}: {os.path.basename(file_path)}")
        print(ds.dims)  # Show dataset dimensions
        print("-" * 40)  # Add a separator for clarity

        
        # Close the dataset to free space in memory
        ds.close()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")


Dataset 1: fairCRPSS_seas5_EFAS_01_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 2: fairCRPSS_seas5_EFAS_02_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 3: fairCRPSS_seas5_EFAS_03_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 4: fairCRPSS_seas5_EFAS_04_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 5: fairCRPSS_seas5_EFAS_05_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 6: fairCRPSS_seas5_EFAS_06_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 7: fairCRPSS_seas5_E

#### Combine Datasets

In [204]:

folder_path = r"C:\Users\giann\OneDrive\Desktop\Thesis\Copernicus_Data\Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2021-2023"

# Get a list of .nc files from the folder
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".nc")]

# Define a function to prepare the dataset
def preprocess(ds):
    # Make 'longitude' and 'latitude' coordinates if they're not already
    if "longitude" not in ds.coords and "longitude" in ds.variables:
        ds = ds.set_coords("longitude")
    if "latitude" not in ds.coords and "latitude" in ds.variables:
        ds = ds.set_coords("latitude")

    # Add a 'step' dimension if it's missing
    if "step" not in ds.sizes and "time" in ds.sizes:
        # Create a dummy 'step' dimension based on time or sequential numbers
        ds["step"] = ("time", np.arange(ds.sizes["time"]))
        ds = ds.swap_dims({"time": "step"})  # Use 'step' as the main dimension

    # Adjust 'step' values if they exist
    if "step" in ds.coords:
        # Convert 'step' values to days
        step_values = ds["step"].values
        step_values_in_days = (step_values - step_values.min()) / (60 * 60 * 24 * 1e9)
        ds = ds.assign_coords(step=("step", step_values_in_days.astype("float64")))

        # Sort 'step' and remove duplicate values
        ds = ds.sortby("step")
        _, index = np.unique(ds["step"], return_index=True)
        ds = ds.isel(step=index)

    return ds


# Combine datasets
def combine_datasets(files):
    try:
        # Open and merge files using xarray
        combined = xr.open_mfdataset(
            files,
            preprocess=preprocess,  # Clean each file before merging
            combine="by_coords",    # Match data using their coordinates
            chunks={"step": 10}     # Use chunks to process faster
        )
        print("Datasets combined successfully!")
        return combined
    except Exception as e:
        print(f"Error combining datasets: {e}")
        return None

# Combine files in two parts for testing
# First part
ds_combined1 = combine_datasets(file_list[:16])  # Use the first 16 files
if ds_combined1:
    print(ds_combined1)

# Second part
ds_combined2 = combine_datasets(file_list[16:])  # Use the remaining files
if ds_combined2:
    print(ds_combined2)


Datasets combined successfully!
<xarray.Dataset> Size: 5GB
Dimensions:                       (time: 1, y: 950, x: 1000, step: 24,
                                   member: 51)
Coordinates:
  * time                          (time) datetime64[ns] 8B 2016-01-01
  * y                             (y) int32 4kB 5497500 5492500 ... 752500
  * x                             (x) int32 4kB 2502500 2507500 ... 7497500
  * step                          (step) float64 192B -8.031e+04 ... 9.956e+04
    latitude                      (y, x) float32 4MB dask.array<chunksize=(950, 1000), meta=np.ndarray>
    longitude                     (y, x) float32 4MB dask.array<chunksize=(950, 1000), meta=np.ndarray>
    number                        (member) int64 408B dask.array<chunksize=(51,), meta=np.ndarray>
    valid_time                    (step) datetime64[ns] 192B dask.array<chunksize=(7,), meta=np.ndarray>
Dimensions without coordinates: member
Data variables:
    fairCRPSS                     (time, st

In [143]:
file_list[15:18]

['C:\\Users\\giann\\OneDrive\\Desktop\\Thesis\\Copernicus_Data\\Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2021-2023\\rdis_seas5_EFAS_20210201_v1.nc',
 'C:\\Users\\giann\\OneDrive\\Desktop\\Thesis\\Copernicus_Data\\Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2021-2023\\rdis_seas5_EFAS_20210301_v1.nc',
 'C:\\Users\\giann\\OneDrive\\Desktop\\Thesis\\Copernicus_Data\\Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2021-2023\\rdis_seas5_EFAS_20210401_v1.nc']

### Climate_indicators_for_Europe_from_2013_to_2023_derived_from_reanalysis

#### Combine Datasets

In [170]:
folder_path = r"C:\Users\giann\OneDrive\Desktop\Thesis\Copernicus_Data\Climate_indicators_for_Europe_from_2013_to_2023_derived_from_reanalysis"

# Find all .nc files in the folder
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".nc")]

# Open and combine multiple datasets using nested merging
combined_dataset = xr.open_mfdataset(file_list, combine='nested', concat_dim='step', coords='minimal')
# Combine files in order
# Merge along the 'step' dimension
# Use only needed coordinates

print("Datasets combined successfully!")


Datasets combined successfully!


#### See Dimension of Dataset

In [155]:
folder_path = r"C:\Users\giann\OneDrive\Desktop\Thesis\Copernicus_Data\Multi-model_seasonal_reforecasts_of_river_discharge_for Europe_2021-2023"

# Get the list of all .nc files in the folder
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".nc")]

# Loop through each file and print its dimensions
for idx, file_path in enumerate(file_list):
    try:
        # Open the dataset
        ds = xr.open_dataset(file_path)
        
        # Print the Dataset name and its dimensions
        print(f"Dataset {idx + 1}: {os.path.basename(file_path)}")
        print(ds.dims)  # Show dataset dimensions
        print("-" * 40)  # Add a separator for clarity
        
         # Close the dataset to save memory
        ds.close()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

Dataset 1: fairCRPSS_seas5_EFAS_01_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 2: fairCRPSS_seas5_EFAS_02_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 3: fairCRPSS_seas5_EFAS_03_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 4: fairCRPSS_seas5_EFAS_04_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 5: fairCRPSS_seas5_EFAS_05_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 6: fairCRPSS_seas5_EFAS_06_v1.nc
FrozenMappingWarningOnValuesAccess({'time': 1, 'step': 7, 'y': 950, 'x': 1000})
----------------------------------------
Dataset 7: fairCRPSS_seas5_E